In [14]:
from __future__ import print_function
from __future__ import unicode_literals

import itertools
import codecs
from os import path
import csv

In [15]:
def load_MSRP(filename):
    
    with codecs.open(filename,'r', b"utf-8" ) as fh:
        nlines = 0
        for line in fh.readlines():
            nlines+=1
            if nlines==1:
                continue
            isparaphrase, id1, id2, str1, str2 = line.split("\t") #the quality fielld is 1 for phraphrases and 0 for not
            if int(isparaphrase)==1:
                yield((int(id1),str1.strip()),(int(id2),str2.strip()))

            
            
                

In [16]:
def create_eval_corpora(base_paraphrases, folder):
    """We want to create the baseline corpus as a new line seperated sentences, so that it works well with Sorchers URAE system.
    Thus all the metadata is stored in seperate files referencing the line numbers"""
    
    phrase_line_num = 0
    with codecs.open(path.join(folder, "phrases.txt"),'w', b"utf-8" ) as phrases_fh:
        with open(path.join(folder, "microsoft_ids.txt"),'w') as microsoft_ids_fh:
            microsoft_ids_csv = csv.writer(microsoft_ids_fh)
            microsoft_ids_csv.writerow(["phrase_line_number","microsoft_id"])
            
            with open(path.join(folder, "paraphases.txt"),"w") as paraphases_fh:
                paraphases_csv = csv.writer(paraphases_fh)
                paraphases_csv.writerow(["phrase_line_num", "paraphase_line_num"])
                 
                for ((m_id1,phrase1),(m_id2,phrase2)) in base_paraphrases:
                    
                    #Add the phrases
                    phrases_fh.write(phrase1)
                    phrases_fh.write("\n")
                    phrase_line_num+=1
                    ln1 = phrase_line_num
                    
                    phrases_fh.write(phrase2)
                    phrases_fh.write("\n")
                    phrase_line_num+=1
                    ln2 = phrase_line_num
                    
                    
                    #add to the record of microsoft ids
                    microsoft_ids_csv.writerow([ln1,m_id1])
                    microsoft_ids_csv.writerow([ln2,m_id2])
                    
                    #add the paraphases, in both directions
                    paraphases_csv.writerow([ln1,ln2])
                    paraphases_csv.writerow([ln2,ln1])
                    
                
        

In [17]:
corp_gen = load_MSRP("base_corpora/MSRP/msr_paraphrase_test.txt")
create_eval_corpora(corp_gen,"prepared_corpora/msrp")